In [ ]:
 ! pip install -q kaggle
 ! mkdir ~/.kaggle
 ! cp /content/kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download datasnaek/mbti-type

 45% 11.0M/24.4M [00:00<00:00, 114MB/s]
100% 24.4M/24.4M [00:00<00:00, 160MB/s]


In [ ]:
!unzip /content/mbti-type.zip

Archive:  /content/mbti-type.zip
  inflating: mbti_1.csv              


In [ ]:
!pip install contractions
import contractions
def fix_contractions(df, column_name = "posts", new_column="cleaned_post"):
    df[new_column] = df[column_name].apply(lambda x: contractions.fix(x))
    return df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.0 MB/s eta 0:00:00


In [ ]:
import re
def clean_data(df, column_name = "cleaned_post"):
    df[column_name] = df[column_name].apply(lambda x: x.lower())
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'@([a-zA-Z0-9_]{1,50})', '', x))
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'#([a-zA-Z0-9_]{1,50})', '', x))
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'http[s]?://\S+', '', x))
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'[^A-Za-z]+', ' ', x))
    df[column_name] = df[column_name].apply(lambda x: re.sub(r' +', ' ', x))
    df[column_name] = df[column_name].apply(lambda x: " ".join([word for word in x.split() if not len(word) <3]))
    return df

In [ ]:
!pip install nltk
import nltk 
nltk.download("all")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
from nltk.tokenize import word_tokenize
def remove_stopwords(data, stopword_list, column="cleaned_post"):
    data[column] = data[column].apply(word_tokenize)
    data[column] = data[column].apply(lambda x: [word for word in x if not word in stopword_list])
    return data

In [ ]:
from nltk.stem import WordNetLemmatizer
def apply_lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [ ]:
def lemmatize(data, stopword_list, column="cleaned_post"):
    data[column] = data[column].apply(apply_lemmatization)
    data[column] = data[column].apply(" ".join)
    return data

In [ ]:
from nltk.corpus import stopwords
stopword_list = stopwords.words("english")

In [ ]:
import pandas as pd

data = pd.read_csv("mbti_1.csv")

In [ ]:
data = fix_contractions(data)
data = clean_data(data)
data = remove_stopwords(data, stopword_list)
data = lemmatize(data, stopword_list)

In [ ]:
data.head()

,type,posts,cleaned_post
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,intj moment sportscenter top ten play prank li...
1,ENTP,'I'm finding the lack of me in these posts ver...,finding lack post alarming sex boring position...
2,INTP,'Good one _____ https://www.youtube.com/wat...,good one course say know blessing curse absolu...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",dear intp enjoyed conversation day esoteric ga...
4,ENTJ,'You're fired.|||That's another silly misconce...,fired another silly misconception approaching ...


In [ ]:
training_data = data[["cleaned_post", "type"]].copy()
training_data.head(5)

,cleaned_post,type
0,intj moment sportscenter top ten play prank li...,INFJ
1,finding lack post alarming sex boring position...,ENTP
2,good one course say know blessing curse absolu...,INTP
3,dear intp enjoyed conversation day esoteric ga...,INTJ
4,fired another silly misconception approaching ...,ENTJ


In [ ]:
from sklearn.model_selection import train_test_split

X = training_data[["cleaned_post"]]
y = training_data[["type"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def divide_types(df):
    df["E-I"] = ""
    df["N-S"] = ""
    df["F-T"] = ""
    df["J-P"] = ""
    for index, row in df.iterrows():
        row["E-I"] = "E" if row.type[0] == "E" else "I"
        row["N-S"] = "N" if row.type[1] == "N" else "S"
        row["F-T"] = "F" if row.type[2] == "F" else "T"
        row["J-P"] = "J" if row.type[3] == "J" else "P"
    return df

y_train = divide_types(y_train)
y_test = divide_types(y_test)

In [ ]:
def make_dummies(data, columns=["E-I", "N-S", "F-T", "J-P"]):
    for column in columns:
        temp_dummy = pd.get_dummies(data[column], prefix="type")
        a = "type_" + column[2]
        b = "type_" + column[0]
        del temp_dummy[a]
        # print(temp_dummy)
        data = data.join(temp_dummy)
    return data
y_train = make_dummies(y_train)
y_test = make_dummies(y_test)


In [ ]:
y_train.head()

,type,E-I,N-S,F-T,J-P,type_E,type_N,type_F,type_J
4080,INFJ,I,N,F,J,0,1,1,1
2614,ENFP,E,N,F,P,1,1,1,0
5414,ENTP,E,N,T,P,1,1,0,0
1039,ENFP,E,N,F,P,1,1,1,0
8294,ENTP,E,N,T,P,1,1,0,0


In [ ]:
X = X_train[["cleaned_post"]]


In [ ]:
y_train

,type,E-I,N-S,F-T,J-P,type_E,type_N,type_F,type_J
4080,INFJ,I,N,F,J,0,1,1,1
2614,ENFP,E,N,F,P,1,1,1,0
5414,ENTP,E,N,T,P,1,1,0,0
1039,ENFP,E,N,F,P,1,1,1,0
8294,ENTP,E,N,T,P,1,1,0,0
...,...,...,...,...,...,...,...,...,...
5734,INFP,I,N,F,P,0,1,1,0
5191,INFP,I,N,F,P,0,1,1,0
5390,INFJ,I,N,F,J,0,1,1,1
860,INFP,I,N,F,P,0,1,1,0


In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler()

In [ ]:
y_ei = y_train[["type_E"]]
y_ns = y_train[["type_N"]]
y_ft = y_train[["type_F"]]
y_jp = y_train[["type_J"]]

In [ ]:
X_over_ei, y_over_ei = oversample.fit_resample(X, y_ei)
X_over_ns, y_over_ns = oversample.fit_resample(X, y_ns)
X_over_ft, y_over_ft = oversample.fit_resample(X, y_ft)
X_over_jp, y_over_jp = oversample.fit_resample(X, y_jp)

In [ ]:
X

,cleaned_post
4080,loved light see anthony doerr pretty recent bo...
2614,depends care fight get really aggressive thing...
5414,welcome home sonny laughing strong tendency to...
1039,really cool like anyone anything help others g...
8294,duck named zeus see trait mammalian predator u...
...,...
5734,cat chihuahua pug mix really hard say prefer k...
5191,ever since remember suffered lived anxiety nev...
5390,known couple infj guy seem lack better term ea...
860,even loner get lonely feel like hard hard rela...


In [ ]:
y_test

,type,E-I,N-S,F-T,J-P,type_E,type_N,type_F,type_J
2802,INTP,I,N,T,P,0,1,0,0
2166,INTJ,I,N,T,J,0,1,0,1
1919,INTP,I,N,T,P,0,1,0,0
360,ENFP,E,N,F,P,1,1,1,0
1115,ENTJ,E,N,T,J,1,1,0,1
...,...,...,...,...,...,...,...,...,...
7023,ISFJ,I,S,F,J,0,0,1,1
6696,INTP,I,N,T,P,0,1,0,0
6746,INFJ,I,N,F,J,0,1,1,1
7966,ISTJ,I,S,T,J,0,0,0,1


In [ ]:
import pandas as pd

df = pd.read_csv('/content/mbti_1.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip


--2023-04-21 06:59:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-21 06:59:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-21 06:59:43--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [ ]:
from keras.layers import LSTM
def Build_Model_RCNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50):

    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector



    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
X_train_ei = X.cleaned_post.values
y_train_ei = y_ei.type_E.values
X_test_ei = X_test.cleaned_post.values
y_test_ei = y_test.type_E.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_ei.tolist()),(X_test_ei.tolist()))

Found 87613 unique tokens.
(8675, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN.summary()

model_RCNN.fit(X_train_Glove, y_train_ei,
                              validation_data=(X_test_Glove, y_test_ei),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

predicted_wei = model_RCNN.predict(X_test_Glove)

predicted_wei = np.argmax(predicted_wei, axis=1)
print("Without oversample ei")
print(metrics.classification_report(y_test_ei, predicted_wei))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_2 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_8 (Conv1D)           (None, 499, 256)          25856     
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 249, 256)         0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 248, 256)          131328    
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 124, 256)         0         
 1D)                                                  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(metrics.f1_score(y_test_ei, predicted_wei))
print(metrics.accuracy_score(y_test_ei, predicted_wei))
print(metrics.precision_score(y_test_ei, predicted_wei))
print(metrics.recall_score(y_test_ei, predicted_wei))

0.0
0.7798270893371758
0.0
0.0


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
X_train_jp = X.cleaned_post.values
y_train_jp = y_jp.type_J.values
X_test_jp = X_test.cleaned_post.values
y_test_jp = y_test.type_J.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_jp.tolist()),(X_test_jp.tolist()))

Found 87613 unique tokens.
(8675, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN.summary()

model_RCNN.fit(X_train_Glove, y_train_jp,
                              validation_data=(X_test_Glove, y_test_jp),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

predicted_wjp = model_RCNN.predict(X_test_Glove)

predicted_wjp = np.argmax(predicted_wjp, axis=1)
print("Without oversample jp")
print(metrics.classification_report(y_test_jp, predicted_wjp))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_3 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_12 (Conv1D)          (None, 499, 256)          25856     
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 249, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 248, 256)          131328    
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 124, 256)         0         
 g1D)                                                 

In [ ]:
print(metrics.f1_score(y_test_jp, predicted_wjp))
print(metrics.accuracy_score(y_test_jp, predicted_wjp))
print(metrics.precision_score(y_test_jp, predicted_wjp))
print(metrics.recall_score(y_test_jp, predicted_wjp))

0.6889580093312597
0.7694524495677233
0.7179902755267423
0.6621823617339312


In [ ]:
X_train_ft = X.cleaned_post.values
y_train_ft = y_ft.type_F.values
X_test_ft = X_test.cleaned_post.values
y_test_ft = y_test.type_F.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_ft.tolist()),(X_test_ft.tolist()))

Found 87613 unique tokens.
(8675, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN.summary()

model_RCNN.fit(X_train_Glove, y_train_ft,
                              validation_data=(X_test_Glove, y_test_ft),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

predicted_wft = model_RCNN.predict(X_test_Glove)

predicted_wft = np.argmax(predicted_wft, axis=1)
print("Without oversample ft")
print(metrics.classification_report(y_test_ft, predicted_wft))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_4 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_16 (Conv1D)          (None, 499, 256)          25856     
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 249, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 248, 256)          131328    
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 124, 256)         0         
 g1D)                                                 

In [ ]:
print(metrics.f1_score(y_test_ft, predicted_wft))
print(metrics.accuracy_score(y_test_ft, predicted_wft))
print(metrics.precision_score(y_test_ft, predicted_wft))
print(metrics.recall_score(y_test_ft, predicted_wft))

0.8305613305613305
0.8121037463976946
0.8095238095238095
0.8527214514407684


In [ ]:
X_train_ns = X.cleaned_post.values
y_train_ns = y_ns.type_N.values
X_test_ns = X_test.cleaned_post.values
y_test_ns = y_test.type_N.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_ns.tolist()),(X_test_ns.tolist()))

Found 87613 unique tokens.
(8675, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN.summary()

model_RCNN.fit(X_train_Glove, y_train_ns,
                              validation_data=(X_test_Glove, y_test_ns),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

predicted_wns = model_RCNN.predict(X_test_Glove)

predicted_wns = np.argmax(predicted_wns, axis=1)
print("without oversample ns")
print(metrics.classification_report(y_test_ns, predicted_wns))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_5 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_20 (Conv1D)          (None, 499, 256)          25856     
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 249, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_21 (Conv1D)          (None, 248, 256)          131328    
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 124, 256)         0         
 g1D)                                                 

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(metrics.f1_score(y_test_ns, predicted_wns))
print(metrics.accuracy_score(y_test_ns, predicted_wns))
print(metrics.precision_score(y_test_ns, predicted_wns))
print(metrics.recall_score(y_test_ns, predicted_wns))

0.923697270471464
0.8582132564841498
0.8582132564841498
1.0


In [ ]:
E_I = predicted_wei
J_P = predicted_wjp
F_T = predicted_wft
N_S = predicted_wns

In [ ]:
predicted = []
for i in range(len(E_I)):
  s = ""
  #enfj
  if(E_I[i] == 0) :
    s = s + "I"
  else :
    s = s + "E"
  if (N_S[i] == 0) :
    s = s + "S"
  else:
    s = s + "N"
  if (F_T[i] == 0) :
    s = s + "T"
  else:
    s = s + "F"
  if (J_P[i] == 0) :
    s = s + "P"
  else:
    s = s + "J"
  predicted.append(s)
print(predicted)

['INTP', 'INFJ', 'INTP', 'INFP', 'INTJ', 'INFJ', 'INFJ', 'INTP', 'INTJ', 'INTP', 'INFP', 'INTJ', 'INFJ', 'INFP', 'INTP', 'INTP', 'INTP', 'INTP', 'INFP', 'INTJ', 'INFJ', 'INTJ', 'INTJ', 'INFP', 'INTP', 'INTJ', 'INTP', 'INTJ', 'INTP', 'INTP', 'INTJ', 'INTJ', 'INTP', 'INFP', 'INTP', 'INFP', 'INFJ', 'INFJ', 'INTP', 'INFP', 'INTJ', 'INFP', 'INTP', 'INFP', 'INFP', 'INFJ', 'INFP', 'INTP', 'INFJ', 'INTP', 'INFP', 'INTP', 'INTP', 'INFP', 'INTJ', 'INFP', 'INTP', 'INFJ', 'INFP', 'INTP', 'INTJ', 'INFP', 'INTP', 'INTP', 'INTP', 'INTP', 'INTP', 'INFP', 'INFP', 'INTP', 'INFP', 'INTP', 'INTP', 'INTP', 'INTJ', 'INFP', 'INFP', 'INTJ', 'INFP', 'INTJ', 'INTP', 'INTJ', 'INTJ', 'INFP', 'INFP', 'INTJ', 'INFJ', 'INFJ', 'INTP', 'INFP', 'INTP', 'INFP', 'INTP', 'INFP', 'INFJ', 'INTJ', 'INTP', 'INFJ', 'INFJ', 'INFP', 'INTP', 'INFJ', 'INFJ', 'INFJ', 'INTP', 'INTP', 'INTP', 'INTP', 'INTP', 'INTP', 'INFJ', 'INTJ', 'INFP', 'INFP', 'INFP', 'INFP', 'INFJ', 'INFP', 'INFJ', 'INFP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'INTJ',

In [ ]:
t = 0
yy = y_test.type.values
for i in range(len(predicted)):
  if(predicted[i] == yy[i]):
    t = t+1
print(t/len(predicted))

0.4270893371757925


In [ ]:
X_train_ei = X_over_ei.cleaned_post.values
y_train_ei = y_over_ei.type_E.values
X_test_ei = X_test.cleaned_post.values
y_test_ei = y_test.type_E.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_ei.tolist()),(X_test_ei.tolist()))

Found 87613 unique tokens.
(12381, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN_ei = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN_ei.summary()

model_RCNN_ei.fit(X_train_Glove, y_train_ei,
                              validation_data=(X_test_Glove, y_test_ei),
                              epochs=15,
                              batch_size=128,
                              verbose=2)
model_RCNN_ei.save('/content/drive/MyDrive/model_RCNN_ei.h5')

predicted_ei = model_RCNN_ei.predict(X_test_Glove)

predicted_ei = np.argmax(predicted_ei, axis=1)
print("With oversample ei")
print(metrics.classification_report(y_test_ei, predicted_ei))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_6 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_24 (Conv1D)          (None, 499, 256)          25856     
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 249, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, 248, 256)          131328    
                                                                 
 max_pooling1d_25 (MaxPoolin  (None, 124, 256)         0         
 g1D)                                                 

In [ ]:
print(metrics.f1_score(y_test_ei, predicted_ei))
print(metrics.accuracy_score(y_test_ei, predicted_ei))
print(metrics.precision_score(y_test_ei, predicted_ei))
print(metrics.recall_score(y_test_ei, predicted_ei))

0.6475138121546962
0.8161383285302594
0.5602294455066922
0.7670157068062827


In [ ]:
X_train_jp = X_over_jp.cleaned_post.values
y_train_jp = y_over_jp.type_J.values
X_test_jp = X_test.cleaned_post.values
y_test_jp = y_test.type_J.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_jp.tolist()),(X_test_jp.tolist()))

Found 87613 unique tokens.
(10085, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN_jp = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN_jp.summary()

model_RCNN_jp.fit(X_train_Glove, y_train_jp,
                              validation_data=(X_test_Glove, y_test_jp),
                              epochs=25,
                              batch_size=128,
                              verbose=2)

model_RCNN_jp.save('/content/drive/MyDrive/model_RCNN_jp.h5')

predicted_jp = model_RCNN_jp.predict(X_test_Glove)

predicted_jp = np.argmax(predicted_jp, axis=1)
print("With oversample jp")
print(metrics.classification_report(y_test_jp, predicted_jp))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_7 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_28 (Conv1D)          (None, 499, 256)          25856     
                                                                 
 max_pooling1d_28 (MaxPoolin  (None, 249, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_29 (Conv1D)          (None, 248, 256)          131328    
                                                                 
 max_pooling1d_29 (MaxPoolin  (None, 124, 256)         0         
 g1D)                                                 

In [ ]:
print(metrics.f1_score(y_test_jp, predicted_jp))
print(metrics.accuracy_score(y_test_jp, predicted_jp))
print(metrics.precision_score(y_test_jp, predicted_jp))
print(metrics.recall_score(y_test_jp, predicted_jp))

0.640582347588717
0.7723342939481268
0.8186046511627907
0.5261584454409567


In [ ]:
X_train_ft = X_over_ft.cleaned_post.values
y_train_ft = y_over_ft.type_F.values
X_test_ft = X_test.cleaned_post.values
y_test_ft = y_test.type_F.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_ft.tolist()),(X_test_ft.tolist()))

Found 87613 unique tokens.
(9249, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN_ft = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN_ft.summary()

model_RCNN_ft.fit(X_train_Glove, y_train_ft,
                              validation_data=(X_test_Glove, y_test_ft),
                              epochs=15,
                              batch_size=128,
                              verbose=2)
model_RCNN_ft.save('/content/drive/MyDrive/model_RCNN_ft.h5')

predicted_ft = model_RCNN_ft.predict(X_test_Glove)

predicted_ft = np.argmax(predicted_ft, axis=1)
print("With oversample ft")
print(metrics.classification_report(y_test_ft, predicted_ft))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_8 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_32 (Conv1D)          (None, 499, 256)          25856     
                                                                 
 max_pooling1d_32 (MaxPoolin  (None, 249, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_33 (Conv1D)          (None, 248, 256)          131328    
                                                                 
 max_pooling1d_33 (MaxPoolin  (None, 124, 256)         0         
 g1D)                                                 

In [ ]:
print(metrics.f1_score(y_test_ft, predicted_ft))
print(metrics.accuracy_score(y_test_ft, predicted_ft))
print(metrics.precision_score(y_test_ft, predicted_ft))
print(metrics.recall_score(y_test_ft, predicted_ft))

0.839231547017189
0.8167146974063401
0.7973102785782901
0.8858057630736392


In [ ]:
X_train_ns = X_over_ns.cleaned_post.values
y_train_ns = y_over_ns.type_N.values
X_test_ns = X_test.cleaned_post.values
y_test_ns = y_test.type_N.values
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer((X_train_ns.tolist()),(X_test_ns.tolist()))

Found 87613 unique tokens.
(13713, 500)
Total 400000 word vectors.


In [ ]:
model_RCNN_ns = Build_Model_RCNN_Text(word_index,embeddings_index, 2)


model_RCNN_ns.summary()

model_RCNN_ns.fit(X_train_Glove, y_train_ns,
                              validation_data=(X_test_Glove, y_test_ns),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

model_RCNN_ns.save('/content/drive/MyDrive/model_RCNN_ns.h5')

predicted_ns = model_RCNN_ns.predict(X_test_Glove)

predicted_ns = np.argmax(predicted_ns, axis=1)
print("With oversample ns")
print(metrics.classification_report(y_test_ns, predicted_ns))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 500, 50)           4380700   
                                                                 
 dropout_9 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_36 (Conv1D)          (None, 499, 256)          25856     
                                                                 
 max_pooling1d_36 (MaxPoolin  (None, 249, 256)         0         
 g1D)                                                            
                                                                 
 conv1d_37 (Conv1D)          (None, 248, 256)          131328    
                                                                 
 max_pooling1d_37 (MaxPoolin  (None, 124, 256)         0         
 g1D)                                                 

In [ ]:
print(metrics.f1_score(y_test_ns, predicted_ns))
print(metrics.accuracy_score(y_test_ns, predicted_ns))
print(metrics.precision_score(y_test_ns, predicted_ns))
print(metrics.recall_score(y_test_ns, predicted_ns))

0.9395255596391581
0.8956772334293948
0.9348404255319149
0.9442578912021491


In [ ]:
E_I = predicted_ei
J_P = predicted_jp
F_T = predicted_ft
N_S = predicted_ns

In [ ]:
predicted = []
for i in range(len(E_I)):
  s = ""
  #enfj
  if(E_I[i] == 0) :
    s = s + "I"
  else :
    s = s + "E"
  if (N_S[i] == 0) :
    s = s + "S"
  else:
    s = s + "N"
  if (F_T[i] == 0) :
    s = s + "T"
  else:
    s = s + "F"
  if (J_P[i] == 0) :
    s = s + "P"
  else:
    s = s + "J"
  predicted.append(s)
print(predicted)

['ENTP', 'INFJ', 'INTP', 'ENFP', 'INTJ', 'INFJ', 'ENFJ', 'INTP', 'ENTP', 'INTP', 'ENFP', 'INTP', 'ENFJ', 'ENFP', 'ISTP', 'INFP', 'ENTP', 'INTP', 'INFP', 'INFP', 'ENFP', 'INTP', 'ENTJ', 'ENFP', 'INTP', 'INTJ', 'INTP', 'INTJ', 'INTP', 'ENTP', 'INTP', 'ENTP', 'INTP', 'ISFP', 'ENFP', 'INTP', 'INFJ', 'ENFJ', 'INTP', 'INFP', 'INTJ', 'ENFP', 'INTP', 'INFP', 'ENFP', 'ENFP', 'INFP', 'ISFP', 'ENFP', 'INTJ', 'INFJ', 'ENTP', 'INTP', 'INFP', 'ISTJ', 'INFP', 'ENTP', 'INFJ', 'INFP', 'ENTP', 'ENTJ', 'INFP', 'ESTP', 'INTP', 'ENTP', 'ISTP', 'INTP', 'INFP', 'INFP', 'INTP', 'INFP', 'ENFP', 'INTP', 'ENTP', 'ENTP', 'INFP', 'INFP', 'ENTP', 'INFP', 'INTJ', 'INTP', 'ENFJ', 'INFJ', 'INTP', 'INFP', 'ENTJ', 'ISFP', 'INTJ', 'INTP', 'ENFP', 'INTP', 'ENFP', 'INTP', 'INFP', 'ENFP', 'INTJ', 'INTP', 'INFJ', 'ENFJ', 'INFP', 'INFP', 'ISFJ', 'ISFJ', 'INFJ', 'ENTP', 'ENTP', 'ESTP', 'INTP', 'INTP', 'INTP', 'INFP', 'INTJ', 'ENFP', 'ENFP', 'INFP', 'INFP', 'INFJ', 'INFP', 'INFJ', 'INFP', 'ENFJ', 'INFP', 'INTP', 'INTP', 'INTP',

In [ ]:
print(len(predicted))

1735


In [ ]:
t = 0
yy = y_test.type.values
for i in range(len(predicted)):
  if(predicted[i] == yy[i]):
    t = t+1
print(t/len(predicted))

0.5054755043227666
